In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from lifetimes import BetaGeoFitter, GammaGammaFitter
from lifetimes.utils import (
    summary_data_from_transaction_data,
    calibration_and_holdout_data,
)
from lifetimes.plotting import plot_cumulative_transactions

In [4]:
plt.style.use("ggplot")
sns.set_palette("husl")

# =======================================================
# 步骤 0: 数据加载
# =======================================================
file_path = "/Users/sousekilyu/Downloads/online_retail_II.xlsx"

print(">> 步骤 0: 正在加载数据...")
# 读取两个 Sheet 并合并
df_0910 = pd.read_excel(file_path, sheet_name="Year 2009-2010")
df_1011 = pd.read_excel(file_path, sheet_name="Year 2010-2011")
df_raw = pd.concat([df_0910, df_1011])
df_raw.head

>> 步骤 0: 正在加载数据...


<bound method NDFrame.head of        Invoice StockCode  ... Customer ID         Country
0       489434     85048  ...     13085.0  United Kingdom
1       489434    79323P  ...     13085.0  United Kingdom
2       489434    79323W  ...     13085.0  United Kingdom
3       489434     22041  ...     13085.0  United Kingdom
4       489434     21232  ...     13085.0  United Kingdom
...        ...       ...  ...         ...             ...
541905  581587     22899  ...     12680.0          France
541906  581587     23254  ...     12680.0          France
541907  581587     23255  ...     12680.0          France
541908  581587     22138  ...     12680.0          France
541909  581587      POST  ...     12680.0          France

[1067371 rows x 8 columns]>

In [5]:
# =======================================================
# 步骤 1: 数据清洗 (Data Cleaning)
# =======================================================
print(">> 步骤 1: 正在清洗数据...")
df_cleaned = df_raw.copy()

# 剔除无 ID、退货订单(以C开头)、负数数量或价格
df_cleaned = df_cleaned[df_cleaned["Customer ID"].notna()]
df_cleaned = df_cleaned[~df_cleaned["Invoice"].astype(str).str.startswith("C")]
df_cleaned = df_cleaned[(df_cleaned["Quantity"] > 0) & (df_cleaned["Price"] > 0)]

# 计算总金额并转换日期格式
df_cleaned["TotalRevenue"] = df_cleaned["Quantity"] * df_cleaned["Price"]
df_cleaned["InvoiceDate"] = pd.to_datetime(df_cleaned["InvoiceDate"])

print(f"   清洗完成。有效交易记录数: {len(df_cleaned)}")


>> 步骤 1: 正在清洗数据...
   清洗完成。有效交易记录数: 805549
